In [1]:
def generate_three_column_latex_table(kin2_path, kin3_path, kin4_path):
    def parse_file(path):
        with open(path, 'r') as f:
            lines = f.readlines()
        return {k.strip(): v.strip() for k, v in (line.split(',') for line in lines if ',' in line)}

    def valerr(val, err):
        return f"${val} \\pm {err}$"

    def round_str(x, digits=None):
        x = float(x)
        if digits is not None:
            return f"{round(x, digits):.{digits}f}"
        return f"{x}"

    d2, d3, d4 = map(parse_file, (kin2_path, kin3_path, kin4_path))

    latex_map = {
        "N_p": r"N_{p}", "N_n": r"N_{n}", "N_bg": r"N_{\text{bg}}", "N_total": r"N_{\text{total}}",
        "f_p": r"f_p", "f_bg": r"f_{\text{bg}}", "f_n": r"f_n", "f_nitrogen": r"f_{\text{nitrogen}}",
        "f_acc": r"f_{\text{acc}}",
        "A_acc": r"A_{\text{acc}}", "A_p": r"A_p", "A_inelastic": r"A_{\text{inelastic}}",
        "P_beam": r"P_{\text{beam}}", "P_target": r"P_{\text{target}}",
        "N_plus_raw": r"N_{\text{raw}}^+", "N_minus_raw": r"N_{\text{raw}}^-",
        "A_raw": r"A_{\text{raw}}", "A_Phys_wavg": r"A_{\text{phys}}",
        "tau": r"\tau", "epsilon": r"\epsilon", "Q2": r"Q^2", "Px": r"P_x", "Pz": r"P_z",
        "GE/GM * muN": r"\mu_n G_E/G_M"
    }

    rows = []

    def row(label, key, errkey=None, digits=None):
        def fmt(v, e=None):
            return valerr(v, e) if e is not None else f"${v}$"
        def get_vals(k, ek=None):
            if ek:
                return [(round_str(d[k], digits), round_str(d[ek], digits)) for d in (d2, d3, d4)]
            else:
                return [round_str(d[k], digits) for d in (d2, d3, d4)]

        if errkey:
            (v2, e2), (v3, e3), (v4, e4) = get_vals(key, errkey)
            rows.append(f"${label}$ & {fmt(v2,e2)} & {fmt(v3,e3)} & {fmt(v4,e4)} \\\\")
        else:
            v2, v3, v4 = get_vals(key)
            rows.append(f"${label}$ & ${v2}$ & ${v3}$ & ${v4}$ \\\\")

    # LaTeX header
    rows.extend([
        "\\begin{table}[h!]",
        "\\centering",
        "\\caption{Summary of extracted quantities for all three kinematics}",
        "\\resizebox{\\textwidth}{!}{%",
        "\\begin{tabular}{lccc}",
        "\\hline",
        "Quantity & Kin2 & Kin3 & Kin4 \\\\",
        "\\hline"
    ])

    # Round to 0 decimals
    for k in ['N_p', 'N_n', 'N_bg', 'N_total', 'N_plus_raw', 'N_minus_raw']:
        row(latex_map[k], k, digits=0)

    for k in ['f_p', 'f_bg', 'f_n', 'f_nitrogen','f_acc']:
        row(latex_map[k], k, k+'_e')

    for k in ['A_acc', 'A_p', 'A_inelastic']:
        row(latex_map[k], k, k+'_e')

    row(latex_map["P_beam"], "P_beam", "P_beam_e")
    row(latex_map["P_target"], "P_target", "P_target_e")

    # A_raw with rel error
    for d in [d2, d3, d4]:
        A = float(d["A_raw"])
        d["A_raw_err"] = f"{A * float(d['A_raw_rel_error']):.4f}"
    row(latex_map["A_raw"], "A_raw", "A_raw_err")

    # A_phys: two-line row
    def extract_phys_err(d):
        A = float(d["A_Phys_wavg"])
        return f"{A:.4f}", f"{A * float(d['A_phys_rel_sys_error']):.4f}", f"{A * float(d['A_phys_rel_sat_error']):.4f}"

    (v2, s2, t2), (v3, s3, t3), (v4, s4, t4) = map(extract_phys_err, (d2, d3, d4))
    rows.append(f"${latex_map['A_Phys_wavg']}$ & ${v2}$ & ${v3}$ & ${v4}$ \\\\")
    rows.append(f"& $\\pm {s2}$ (stat) $\\pm {t2}$ (sys) & $\\pm {s3}$ (stat) $\\pm {t3}$ (sys) & $\\pm {s4}$ (stat) $\\pm {t4}$ (sys) \\\\")

    for k in ['tau', 'epsilon', 'Q2', 'Px', 'Pz']:
        row(latex_map[k], k)

    # GE/GM with two-line
    ge_vals = [d["GE/GM * muN"] for d in (d2, d3, d4)]
    ges = [d["GE/GM_stat_error"] for d in (d2, d3, d4)]
    gesys = [d["GE/GM_sys_error"] for d in (d2, d3, d4)]
    rows.append(f"${latex_map['GE/GM * muN']}$ & ${ge_vals[0]}$ & ${ge_vals[1]}$ & ${ge_vals[2]}$ \\\\")
    rows.append(f"& $\\pm {ges[0]}$ (stat) $\\pm {gesys[0]}$ (sys) & $\\pm {ges[1]}$ (stat) $\\pm {gesys[1]}$ (sys) & $\\pm {ges[2]}$ (stat) $\\pm {gesys[2]}$ (sys) \\\\")

    # Footer
    rows.extend([
        "\\hline",
        "\\end{tabular}%",
        "}",
        "\\label{tab:kin_summary_all}",
        "\\end{table}"
    ])

    return "\n".join(rows)


In [5]:
latex_code = generate_three_column_latex_table("kin2results.txt", "kin3results.txt", "kin4combinedresults.txt")
print(latex_code)


\begin{table}[h!]
\centering
\caption{Summary of extracted quantities for all three kinematics}
\resizebox{\textwidth}{!}{%
\begin{tabular}{lccc}
\hline
Quantity & Kin2 & Kin3 & Kin4 \\
\hline
$N_{p}$ & $2190$ & $1166$ & $536$ \\
$N_{n}$ & $92413$ & $20560$ & $3989$ \\
$N_{\text{bg}}$ & $7164$ & $16382$ & $12962$ \\
$N_{\text{total}}$ & $101564$ & $38589$ & $17795$ \\
$N_{\text{raw}}^+$ & $51170$ & $19479$ & $8513$ \\
$N_{\text{raw}}^-$ & $47561$ & $18229$ & $8234$ \\
$f_p$ & $0.0216 \pm 0.0005$ & $0.0302 \pm 0.0009$ & $0.0301 \pm 0.0013$ \\
$f_{\text{bg}}$ & $0.0705 \pm 0.0009$ & $0.4245 \pm 0.004$ & $0.6958 \pm 0.0082$ \\
$f_n$ & $0.9099 \pm 0.0041$ & $0.5328 \pm 0.0046$ & $0.1861 \pm 0.0036$ \\
$f_{\text{nitrogen}}$ & $0.0264 \pm 0.0005$ & $0.02 \pm 0.0004$ & $0.0191 \pm 0.0003$ \\
$f_{\text{acc}}$ & $0.0183 \pm 0.0004$ & $0.0332 \pm 0.001$ & $0.041 \pm 0.0016$ \\
$A_{\text{acc}}$ & $-0.0009 \pm 0.0056$ & $0.0147 \pm 0.009$ & $0.0254 \pm 0.0192$ \\
$A_p$ & $0.001 \pm 0.0001$ & $0.00